# Test a reform with Open Fisca

In [1]:
# Activate multi-output in notebook
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
import unittest

import numpy as np
from openfisca_core import periods  # type: ignore
from openfisca_core.parameters import ParameterAtInstant, ParameterNode  # type: ignore
from openfisca_core.periods import instant
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_core.tools import assert_near
from openfisca_france import FranceTaxBenefitSystem  # type: ignore
from openfisca_france.model.base import Reform  # type: ignore

from leximpact_socio_fisca_simu_etat.logger import logger

tc = unittest.TestCase()
from typing import Dict, List

## Basic OpenFisca reform test

In [3]:
# Define a reform
def modifier(parameters):
    reform_period = periods.period(
        "year:1900:200"
    )  # Force the new value from 1900 to 2100.
    parameters.prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux.update(
        start=reform_period.start, value=0.5
    )
    return parameters


class OurReforme(Reform):
    def apply(self):
        self.modify_parameters(modifier_function=modifier)


# Compute reform
period = "2022-01"
legislation_reforme = OurReforme(FranceTaxBenefitSystem())
sb = SimulationBuilder()
simulation = sb.build_default_simulation(legislation_reforme, count=1)
simulation.set_input("salaire_de_base", period, np.array([1_000]))
csg_deductible_salaire = simulation.calculate("csg_imposable_salaire", period)
csg_deductible_salaire

array([-491.25], dtype=float32)

In [4]:
# Assert that result is equal to (salary - 1.75%) - 50%
tc.assertEqual(csg_deductible_salaire[0], -1_000 * (1 - 0.0175) * 0.5)

## More complex reform

### Define parameters

In [5]:
csg_deductible_new = 0.9
csg_abattement = 0.03
PFSS = 3_428
nb_pfss = 4
salaires_de_base = np.array([1000, 3 * PFSS, 4 * PFSS - 1, 4 * PFSS, 4 * PFSS + 1])
period = "2022"

## Compute the values before the reform

In [6]:
print(
    f"prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux={FranceTaxBenefitSystem().parameters(period).prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux}"
)
print(
    f"prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux={FranceTaxBenefitSystem().parameters(period).prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux}"
)

prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux=0.068
prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux=0.024


In [7]:
tbs = FranceTaxBenefitSystem()

sb = SimulationBuilder()
simulation = sb.build_default_simulation(
    FranceTaxBenefitSystem(), count=len(salaires_de_base)
)

simulation.set_input("salaire_de_base", period, salaires_de_base)

assiette_csg_abattue = simulation.calculate_add("assiette_csg_abattue", period)

print("assiette_csg_abattue", assiette_csg_abattue)

print(
    "csg_imposable_salaire", simulation.calculate_add("csg_imposable_salaire", period)
)
print(
    "csg_deductible_salaire", simulation.calculate_add("csg_deductible_salaire", period)
)
csg_avant = simulation.calculate_add("csg", "2022")
print("csg avant", csg_avant)
assert_near(
    csg_avant,
    [-90.39, -929.57074, -1239.3375, -1239.4276, -1239.5198],
    absolute_error_margin=0.01,
)

assiette_csg_abattue [  999.99994 10284.      13710.999   13712.001   13713.     ]
csg_imposable_salaire [ -23.579521 -242.49603  -323.30402  -323.32797  -323.352   ]
csg_deductible_salaire [ -66.80864 -687.07196 -916.02814 -916.0959  -916.1639 ]
csg avant [  -90.38816  -929.568   -1239.3322  -1239.4238  -1239.5159 ]


### Verify that the returned values are good

Here we manually compute the formula of CSG tax on salary to verify the result

In [8]:
expected = []
for salaire in salaires_de_base:
    if salaire >= nb_pfss * PFSS:
        csg = -(nb_pfss * PFSS * (1 - 0.0175)) * (0.068 + 0.024)
        csg += -(salaire - PFSS * nb_pfss) * (0.068 + 0.024)
    else:
        csg = -(salaire * (1 - 0.0175)) * (0.068 + 0.024)
    expected.append(csg)
assert_near(csg_avant, expected, absolute_error_margin=0.01)

## Import reform updater from leximpact_socio_fisca_simu_etat

This class CustomReform allow to input parameters changes using Open Fisca formalism.

In [9]:
from leximpact_socio_fisca_simu_etat.csg_simu import CustomReform
from leximpact_socio_fisca_simu_etat.schema import (
    AllSimulationResult,
    OneSimulationResult,
    ReformeSocioFiscale,
)

[leximpact_socio-fisca-simu-etat DEBUG @ 14:52:32] Connecting to Redis None:6379
[leximpact_socio-fisca-simu-etat WARNING @ 14:52:32] Unable to connect to Redis cache : Error 111 connecting to None:6379. Connection refused.


## Define the reforms

In [10]:
reform_bareme_apres = ReformeSocioFiscale(
    base=2021,
    plf=2022,
    amendement={
        "prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux": {
            "start": "2021-09-01",
            "type": "parameter",
            "value": csg_deductible_new,
        },
        "prelevements_sociaux.contributions_sociales.csg.activite.imposable.abattement": {
            "scale": [
                {"rate": {"value": csg_abattement}, "threshold": {"value": 0}},
                {"rate": {"value": 0}, "threshold": {"value": nb_pfss}},
            ],
            "start": "2020-09-01",
            "type": "scale",
        },
        "prelevements_sociaux.contributions_sociales.csg.activite.deductible.abattement": {
            "scale": [
                {"rate": {"value": csg_abattement}, "threshold": {"value": 0}},
                {"rate": {"value": 0}, "threshold": {"value": nb_pfss}},
            ],
            "start": "2020-09-01",
            "type": "scale",
        },
    },
    output_variables=[],
)

## Apply the reform to the FranceTaxBenefitSystem

In [11]:
ma_reforme_apres = CustomReform(FranceTaxBenefitSystem(), reform_bareme_apres, period)

### Verify the parameters values

In [12]:
print(
    f"prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux={ma_reforme_apres.parameters('2022').prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux}"
)
print(
    f"prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux={ma_reforme_apres.parameters('2022').prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux}"
)

# ma_reforme_apres.parameters(
#     "2020-09"
# ).prelevements_sociaux.contributions_sociales.csg.activite.imposable.abattement

tc.assertEqual(
    ma_reforme_apres.parameters(
        "2022"
    ).prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux,
    csg_deductible_new,
)

tc.assertEqual(
    ma_reforme_apres.parameters(
        "2022"
    ).prelevements_sociaux.contributions_sociales.csg.activite.imposable.abattement.rates[
        0
    ],
    csg_abattement,
)

prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux=0.9
prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux=0.024


## Compute the values after the reform

In [13]:
sb = SimulationBuilder()
simulation = sb.build_default_simulation(ma_reforme_apres, count=len(salaires_de_base))
# activate the trace
simulation.trace = True
simulation.set_input("salaire_de_base", period, salaires_de_base)


assiette_csg_abattue = simulation.calculate_add("assiette_csg_abattue", period)

print("assiette_csg_abattue", assiette_csg_abattue)
print(
    "csg_imposable_salaire", simulation.calculate_add("csg_imposable_salaire", period)
)
print(
    "csg_deductible_salaire", simulation.calculate_add("csg_deductible_salaire", period)
)
csg_apres = simulation.calculate_add("csg", "2022")

print("\ncsg avant", csg_avant)
print("csg apres", csg_apres)


# print calculation steps
# simulation.tracer.print_computation_log()

assiette_csg_abattue [  999.99994 10284.      13710.999   13712.001   13713.     ]
csg_imposable_salaire [ -23.280005 -239.41151  -319.1913   -319.21533  -319.2393  ]
csg_deductible_salaire [  -873.     -8977.933 -11969.676 -11970.575 -11971.476]

csg avant [  -90.38816  -929.568   -1239.3322  -1239.4238  -1239.5159 ]
csg apres [  -896.28   -9217.344 -12288.867 -12289.791 -12290.715]


### Verify that the returned values for the reform are good

In [14]:
expected = []
for salaire in salaires_de_base:
    if salaire >= nb_pfss * PFSS:
        csg = -(nb_pfss * PFSS * (1 - csg_abattement)) * (csg_deductible_new + 0.024)
        csg += -(salaire - PFSS * nb_pfss) * (csg_deductible_new + 0.024)
    else:
        csg = -(salaire * (1 - csg_abattement)) * (csg_deductible_new + 0.024)
    expected.append(csg)
assert_near(csg_apres, expected, absolute_error_margin=0.9)